# Model selection 

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In the previous sprint, we covered several aspects of `ols()` and illustrated them on the basis of the reaction times datatset offered by Gries:

In [ ]:
df = pd.read_csv("../../datasets/gries/05-2_reactiontimes.csv", sep="\t")
df["LENGTH"] = df.CASE.str.len()
# Level our categoricals so statsmodels can see them. Unless we use them in something like difference
# encoding it won't make much model difference, but it can make a lot of interpretability
# difference.
df["IMAGEABILITY"] = df.IMAGEABILITY.astype(
    pd.api.types.CategoricalDtype(categories=["lo", "hi"], ordered=True)
)
df["FAMILIARITY"] = df.FAMILIARITY.astype(
    pd.api.types.CategoricalDtype(categories=["lo", "med", "hi"], ordered=True)
)
df

In [ ]:
preds = df.columns[2:]
preds

We discussed how independent variables (whether they were numeric, binary or categorical) could be used and combined to predict the reaction time as our main dependent variable. We saw how we can also combine these predictors (using a `+` in the formula notation), since they all seem to have some worth in modelling RT. We could even combine them all:

In [ ]:
model = smf.ols(f"RT ~ {' + '.join(preds)}", data=df).fit()
print(model.summary())

Interestingly, as a whole, this model is well below the critical threshold for a statistical significance, but most of the estimates for the parameters now appear to be insignificant (apart from the simple LENGTH of a word). This is confusing and raises the issue: which predictors should we include?

## Parameter Selection

If we have the opportunity to include multiple variables in a linear model, we'll need a way to determine whether it's a good idea to add them. In other words, we'll need strategies for **model selection**, i.e. determining the "best" model for our data. There are basically two options:

1. Forward selection: you start with a small dataset and gradually add more predictors, if these prove "useful".
2. Backward selection: you start out with a huge model that covers many predictors, and you gradually remove predictors that don't appear to be "useful".

Several views exist on this matter. Many people will consider it bad practice to test for predictors that weren't theoretically motivated in the first place, but heeding that advice, it can become difficult to discover new things. Both strategies ultimately have the same goal: "Arrive at a **minimally adequate model**", i.e. the smallest model that gives you an optimal fit. As a rule of thumb, smaller model are *always* to be prefered (cf. Occam's Razor). How do you determine whether a predictor is "useful"? There are two "metrics", that can be used:

1. Look at $p$-values (if they are significant you keep them in). The problem is that you still end up with many variables that are all somewhat significant, but in a very large model.
2. Work with a **criterion based approach**, using for instance the **Akaike Information Criterion** to compare models with and without a predictor. The nice property of AIC is that it penalizes models on the basis of their size (the number of predictors and parameters): the AIC for a model with *many* parameters will be automatically adjusted downwards.

<blockquote>
SIDEBAR AIC and BIC

You can read more then you ever wanted to about the AIC [on wikipedia](https://en.wikipedia.org/wiki/Akaike_information_criterion), but the main things we want you to remember are:
- lower AIC (or BIC) are good
- AIC is only useful to compare **models for the same thing**. It makes no sense as an absolute scale (ie don't compare the AIC for different datasets or different dependent variables)
- AIC (and even more so BIC) already penalize model parameters, so *as a rule of thumb* if adding a parameter increases $R^2$ and decreases AIC it is probably worthwhile
</blockquote>

## Buckaroo Datascience

**For the sake of argument** let's look at a kind of hacky way to compare models. Since our prediction formula can be specified as a string, we can modify the specification string, and refit the model with every combination of parameters. The code below is a little challenging, so just think about the intuition.

We we take *all subsets* from our five predictors, (from one feature to four, and *all combinations at each size*), build a string from that set, and then record a few model criteria. Remember that both AIC and BIC are good ways to compare model performance, and a **lower score is better**. The (adjusted) $R^2$ tells us how much variability is explained, and a **higher score is better**. So, we'll just combine them to create two *ad hoc* measures of Model Goodness, which we call `R2AIC` and `R2BIC`.

In [5]:
from itertools import chain, combinations

res = []
for subset in chain.from_iterable(combinations(preds, r) for r in range(1, len(preds))):
    model = smf.ols(f"RT ~ {' + '.join(subset)}", data=df).fit()
    res.append(
        {
            "model": " + ".join(subset),
            "BIC": model.bic,
            "AIC": model.aic,
            "ADJR2": model.rsquared_adj,
            "R2AIC": 1 / model.aic * model.rsquared_adj,
            "R2BIC": 1 / model.bic * model.rsquared_adj,
        }
    )

This gives us a `list` of `dict` objects, with identical keys. This is one of the (many) ways to build a pandas `DataFrame`.

In [ ]:
res[:3]

Now we can rank our models!

In [ ]:
pd.DataFrame(res).sort_values("R2AIC", ascending=False)

In [ ]:
for x in pd.DataFrame(res).sort_values("R2AIC", ascending=False).model.head(10):
    print(f'"{x}",')

... so the best model **by this ranking** has only three features. Thinking about some sanity checks:
- There are many models with an AIC/BIC in the high 400s, so nothing exceptional is happening (many are much worse)
- The Adjusted $R^2$ for the top model doesn't clearly stand out (it's not even the best)
- This is the best combination, and also one of the most parsimonious 

So it seems fair. The second-ranked model is also worth a very close look -- we would probably need to go on and think hard about our predictors to make a choice. Some things to think about:
- Is there a good theoretical *reason* to include `FAMILIARITY` -- it scored the best of the single predictor models?
    - This can only be answered according to the scientific hypothesis motivating the study!
- Is there 'overlap' between `FAMILIARITY` and `MEANINGFULNESS`? (the two-predictor model `FAMILIARITY + LENGTH` didn't do so well...)

Some people would choose the leaner model on principle; some wouldn't. There is no right or wrong answer, it is a choice that you need to make using your scientific expertise, *and not just by looking at numbers*. Sorry.

Anyway, here are your two leading candidates. Choose your fighter.

In [ ]:
model = smf.ols("RT ~ C(FAMILIARITY,Diff) + MEANINGFULNESS + LENGTH", data=df).fit()
print(model.summary())

In [ ]:
model = smf.ols("RT ~ MEANINGFULNESS + LENGTH", data=df).fit()
print(model.summary())

```
Version History

Current: v1.0.1

7/10/24: 1.0.0: first draft, BN
09/10/24: 1.0.1: proofread, MK
```